In [1]:
import paho.mqtt.client as mqtt
import simpleaudio as sa
import time
import pandas as pd
import json
import numpy as np
import sys
import pipeclient
from warnings import warn

In [6]:
if '':
    print('bla')

In [2]:
use_audacity = True
use_aud_list = True


if use_audacity:
    aud = pipeclient.PipeClient()
else:
    opera = sa.WaveObject.from_wave_file('opera.wav')    

# Define set of functions for start and stop
if use_audacity:
    def play(start=0):
        aud.write('Stop')
        aud.write('CursProjectStart')
        aud.write('PlayStop')
        return time.time()
    def stop():
        aud.write('Stop')
else:
    def play():
        player = opera.play()
        return time.time()

In [3]:
if use_audacity and use_aud_list:
    aud.write('GetInfo: Type=Labels')
    time.sleep(0.2)
    lbl0 = pd.DataFrame(json.loads(aud.read().split('BatchCommand')[0])[0][1])
    lbl0.columns = ['start', 'end', 'label']
else:
    lbl0 = pd.read_csv('opera.txt', sep='\t', header=None)
    lbl0.columns = ['start', 'end', 'label']
    
# Table sanitizer
lbl0 = lbl0.drop(columns='label').join(
    lbl0.label.str.split(
        ',', expand=True).rename(
        columns={0: 'comment', 1: 'speed', 2: 'trigger'}))
lbl0.trigger = lbl0.trigger.str.lower().str.strip() == 't'
lbl0.speed = pd.to_numeric(lbl0.speed).fillna(-1).astype(int)

print('Loaded score table. Starts with:\n', lbl0.head(5))

Sending command: GetInfo: Type=Labels
Loaded score table. Starts with:
       start       end           comment  speed  trigger
0   7.16926   7.16926             slow7      7    False
1  21.33730  21.33730             slow5      5    False
2  31.56140  31.56140             slow3      3    False
3  42.36630  42.36630       BeginnArie1     -1     True
4  72.34020  72.34020  BeginnMotivArie1     -1     True


In [5]:
global waiting, trig_steps
waiting = False
trig_steps = []

def on_message(client, userdata, message):
    global waiting, trig_steps
    if 'WaitForMusic' in message.payload.decode():
        waiting = True
        msg = json.loads(message.payload.decode())
        trig_steps.append(msg['WaitForMusic'])
        print('Waiting in:', msg['WaitForMusic'])

# del client
client = mqtt.Client()
client.on_message = on_message
client.connect('172.31.1.150', 1883)
client.subscribe('music/commands')
client.loop_start()

In [5]:
# client.connect('172.31.1.150', 1883)
t0 = play()
last_msg = client.publish('music/state', json.dumps({'action': 'Trigger'}))
lbl = lbl0.copy()
qt = [] # trigger queue

while len(lbl) > 0: 
    
    try:       
        
        # Any missing Triggers?
        if (len(qt) > 0) and waiting:
            comm = qt.pop(0)
            print(f'SENDING DELAYED TRIGGER {comm}')
            last_msg = client.publish('music/state', json.dumps({'action': 'Trigger'}))
            time.sleep(0.05)
            
        ela = time.time() - t0
        passed = lbl[lbl.start <= ela]
        lbl.drop(passed.index, inplace=True)
        
        for event in passed.itertuples():
            print('Passing Label:', event.comment)
            msg = {}
            if event.trigger:
                if waiting:
                    msg['action'] = 'Trigger'
#                     print(f'Sending trigger {event.comment} to finish step {last_msg[-1]}')
                    waiting = False
                else:
                    # Trigger while not waiting for one...
                    qt.append(event.comment)
#                     warn(f'TRIGGER {event.comment} REQUESTED BY MUSIC WHILE NOT AWAITED ({len(qt)} TRIGGER(s) WAITING). CHECK TIMING!!!')
            
            if event.speed > -1:
                msg['speed'] = event.speed
            
            if len(msg) > 0:
                print('Sending message:', msg)
                last_msg = client.publish('music/state', json.dumps(msg))
                
        time.sleep(0.05)
        
    except KeyboardInterrupt:
        stop()
        client.loop_stop()        
        print('Opera finished.')
        break
        
    except Exception as err:
        stop()
        raise err

Sending command: Stop
Sending command: CursProjectStart
Sending command: PlayStop


NameError: name 'client' is not defined

In [9]:
client.unsubscribe('music/commands')
client.disconnect()

0

In [21]:
stop()

Sending command: Stop


In [61]:
lbl[passed]

,start,end,label
0,5.892005,5.892005,uhhh
1,31.985169,31.985169,ahhh


In [60]:
lbl.drop(passed.index)

,start,end,label


In [ ]:
def send_command('lbl')

In [77]:
player.stop()